In [10]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
import random
from matplotlib import pyplot as plt

In [11]:
maxAcc = 0.0
maxIter = 0
C_Lambda = 1
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
PHI = []
IsSynthetic = False

In [12]:
def CreateDataMatrixSamePair(filePath):
    target = []
    count= 0
    count2 = 0
    addMatrix =[]
    subMatrix = []
    with open(filePath, 'rU') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            if(count<2000):
                target.append(int(row[2]))
                data = []
                data1= []
                with open('GSC_Data.csv', 'rU') as f1:
                    reader1 = csv.reader(f1) 
                    next(reader1)
                    for row1 in reader1:
                        if(row1[0] == row[0]):
                            for column in row1:
                                data.append(column)
                        if(row1[0] == row[1]):
                            for column in row1:
                                data1.append(column)
                            break
                    data=data[1:]
                    data=[int(data) for data in data]
                    data1=data1[1:]
                    data1=[int(data1) for data1 in data1]
                    add = data + data1
                    sub = [abs((data - data1))  for data,data1 in zip(data,data1)]
                    addMatrix.append(add)
                    subMatrix.append(sub)
                count = count +1
    with open("GSC_diffn_pairs.csv", 'rU') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            if(count2) <2000:
                target.append(int(row[2]))
                data = []
                data1= []
                with open('GSC_Data.csv', 'rU') as f1:
                    reader1 = csv.reader(f1) 
                    next(reader1)
                    for row1 in reader1:
                        if(row1[0] == row[0]):
                            for column in row1:
                                data.append(column)
                        if(row1[0] == row[1]):
                            for column in row1:
                                data1.append(column)
                            break
                    data=data[1:]
                    data=[int(data) for data in data]
                    data1=data1[1:]
                    data1=[int(data1) for data1 in data1]
                    add = data + data1
                    sub = [abs((data - data1))  for data,data1 in zip(data,data1)]
                    addMatrix.append(add)
                    subMatrix.append(sub)
            count2 = count2 +1
    return addMatrix , subMatrix, target

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    return BigSigma


def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.pinv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

In [13]:
Add, Subtract, Target =  CreateDataMatrixSamePair('GSC_same_pairs.csv')
shuffledDataMatrix = list(zip(Add, Subtract, Target))
random.shuffle(shuffledDataMatrix)
Add, Subtract, Target = zip(*shuffledDataMatrix)
deladd = (np.where(~np.array(Add).any(axis=0))[0])
delsub = (np.where(~np.array(Subtract).any(axis=0))[0])
Add = np.delete(Add,deladd,axis=1)
Subtract = np.delete(Subtract,delsub,axis=1)
Add = np.transpose(Add)
Subtract = np.transpose(Subtract)
print(np.array(Target).shape)
print(np.array(Add).shape)
print(np.array(Subtract).shape)
#print(Subtract)

D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 'U' mode is deprecated
  from ipykernel import kernelapp as app
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: 'U' mode is deprecated
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: 'U' mode is deprecated


(4000,)
(935, 4000)
(472, 4000)


## Prepare Training Data

In [14]:
TrainingTarget = np.array(GenerateTrainingTarget(Target,TrainingPercent))
TrainingAddData   = GenerateTrainingDataMatrix(Add,TrainingPercent)
TrainingSubtractData   = GenerateTrainingDataMatrix(Subtract,TrainingPercent)
print(TrainingTarget.shape)
print(TrainingAddData.shape)
print(TrainingSubtractData.shape)

(3200,)
(935, 3200)
(472, 3200)


## Prepare Validation Data

In [15]:
ValDataAct = np.array(GenerateValTargetVector(Target,ValidationPercent, (len(TrainingTarget))))
ValDataAdd    = GenerateValData(Add,ValidationPercent, (len(TrainingTarget)))
ValDataSubtract    = GenerateValData(Subtract,ValidationPercent, (len(TrainingTarget)))
print(ValDataAct.shape)
print(ValDataAdd.shape)
print(ValDataSubtract.shape)

(399,)
(935, 399)
(472, 399)


## Prepare Test Data

In [16]:
TestDataAct = np.array(GenerateValTargetVector(Target,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestDataAdd = GenerateValData(Add,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
TestDataSubtract= GenerateValData(Subtract,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
print(TestDataAct.shape)
print(TestDataAdd.shape)
print(TestDataSubtract.shape)

(399,)
(935, 399)
(472, 399)


## Closed Form Solution For Feature Addition

In [17]:
ErmsArr = []
AccuracyArr = []
M= 25
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingAddData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(Add, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(Add, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestDataAdd, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValDataAdd, Mu, BigSigma, 100)

In [18]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(25, 935)
(935, 935)
(3200, 25)
(25,)
(399, 25)
(399, 25)


## Finding Erms on training, validation and test set

In [21]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))

In [22]:
print ("-------Closed Form with Radial Basis Function For Feature Addition-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))
print ("Accuracy Training   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("Accuracy Validation = " + str(float(ValidationAccuracy.split(',')[0])))
print ("Accuracy Testing    = " + str(float(TestAccuracy.split(',')[0])))

-------Closed Form with Radial Basis Function For Feature Addition-------
----------------------------------------------------
E_rms Training   = 0.33376479277618387
E_rms Validation = 0.32797815867113816
E_rms Testing    = 0.35669788307100986
Accuracy Training   = 86.0
Accuracy Validation = 86.96741854636592
Accuracy Testing    = 83.70927318295739


## Gradient Descent solution for Linear Regression For Feature Addition

In [23]:
W_Now        = np.dot(220, W)
La           = 2
learningRate = 0.02
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1601):
    
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    
    #-----------------ValidationData Accuracy---------------------#
VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    
    #-----------------TestingData Accuracy---------------------#
TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
Erms_Test = GetErms(TEST_OUT,TestDataAct)

In [24]:
print ("-------Gradient Descent For Feature Addition-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(Erms_TR.split(',')[1])))
print ("E_rms Validation = " + str(float(Erms_Val.split(',')[1])))
print ("E_rms Testing    = " + str(float(Erms_Test.split(',')[1])))
print ("Accuracy Training   = " + str(float(Erms_TR.split(',')[0])))
print ("Accuracy Validation = " + str(float(Erms_Val.split(',')[0])))
print ("Accuracy Testing    = " + str(float(Erms_Test.split(',')[0])))

-------Gradient Descent For Feature Addition-------
----------------------------------------------------
E_rms Training   = 0.6868774558191151
E_rms Validation = 0.6782772525883317
E_rms Testing    = 0.7411713282792695
Accuracy Training   = 50.71875
Accuracy Validation = 51.8796992481203
Accuracy Testing    = 42.606516290726816


## Closed Form Solution For Feature Subtraction

In [25]:
M= 25

kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingSubtractData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(Subtract, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(Subtract, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestDataSubtract, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValDataSubtract, Mu, BigSigma, 100)

In [26]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(25, 472)
(472, 472)
(3200, 25)
(25,)
(399, 25)
(399, 25)


## Finding Erms on training, validation and test set

In [28]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))

In [29]:
print ("-------Closed Form with Radial Basis Function For Feature Subtraction-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))
print ("Accuracy Training   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("Accuracy Validation = " + str(float(ValidationAccuracy.split(',')[0])))
print ("Accuracy Testing    = " + str(float(TestAccuracy.split(',')[0])))

-------Closed Form with Radial Basis Function For Feature Subtraction-------
----------------------------------------------------
E_rms Training   = 0.28652817769779193
E_rms Validation = 0.2842780720051207
E_rms Testing    = 0.3020517225130166
Accuracy Training   = 91.96875
Accuracy Validation = 92.4812030075188
Accuracy Testing    = 88.22055137844612


##  Gradient Descent solution for Linear Regression For Feature Subtraction

In [30]:
W_Now        = np.dot(220, W)
La           = 0.5
learningRate = 0.02
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1270):
    
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [31]:
print ("-------Gradient Descent For Feature Subtraction-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(Erms_TR.split(',')[1])))
print ("E_rms Validation = " + str(float(Erms_Val.split(',')[1])))
print ("E_rms Testing    = " + str(float(Erms_Test.split(',')[1])))
print ("Accuracy Training   = " + str(float(Erms_TR.split(',')[0])))
print ("Accuracy Validation = " + str(float(Erms_Val.split(',')[0])))
print ("Accuracy Testing    = " + str(float(Erms_Test.split(',')[0])))

-------Gradient Descent For Feature Subtraction-------
----------------------------------------------------
E_rms Training   = 0.5332713440581739
E_rms Validation = 0.5258692861174672
E_rms Testing    = 0.5514124824129272
Accuracy Training   = 54.9375
Accuracy Validation = 55.13784461152882
Accuracy Testing    = 47.11779448621554


In [32]:
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

## Logistic Regression solution for Feature Subtraction

In [33]:
W_Now        = np.zeros(472)
La           = 2
learningRate = 0.05
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1600):
    scores = np.dot(W_Now, TrainingSubtractData)
    prediction  = sigmoid(scores)
    #print ('---------Iteration: ' + str(i) + '--------------')
    error =  prediction - TrainingTarget
    Delta_E_D     = np.dot((TrainingSubtractData), error)/TrainingTarget.size
    #La_Delta_E_W  = np.dot(La,W_Now)
    #Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E_D)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------Training Data Accuracy---------------------#
right=0
wrong=0
TR_TEST=[]
TR_TEST_OUT   =  sigmoid(np.dot(W_Now, TrainingSubtractData)) 
for res in TR_TEST_OUT:
    if (res>0.5):
        TR_TEST.append(1)
    else:
        TR_TEST.append(0)
for i,j in zip(TR_TEST, TrainingTarget):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Training Accuracy: " + str(right/(right+wrong)*100))
     
    #-----------------Validation Data Accuracy---------------------# 
right=0
wrong=0
VAL_TEST=[] 
VAL_TEST_OUT  = sigmoid(np.dot(W_Now, ValDataSubtract))
for res in VAL_TEST_OUT:
    if (res>0.5):
        VAL_TEST.append(1)
    else:
        VAL_TEST.append(0)
for i,j in zip(VAL_TEST, ValDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Validation Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------Testing Data Accuracy---------------------#
right=0
wrong=0
TEST =[]
TEST_OUT =  sigmoid(np.dot(W_Now, TestDataSubtract))  
for res in TEST_OUT:
    if (res>0.5):
        TEST.append(1)
    else:
        TEST.append(0)
for i,j in zip(TEST, TestDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Training Accuracy: 97.46875
Validation Accuracy: 97.24310776942356
Testing Accuracy: 96.2406015037594


## Logistic Regression solution for Feature Addition

In [34]:
W_Now        = np.zeros(935)
La           = 2
learningRate = 0.05
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1600):

    scores = np.dot(W_Now, TrainingAddData)
    prediction  = sigmoid(scores)
    #print ('---------Iteration: ' + str(i) + '--------------')
    error =  prediction - TrainingTarget
    Delta_E_D     = np.dot(TrainingAddData, error)
    #La_Delta_E_W  = np.dot(La,W_Now)
    #Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E_D)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

#-----------------Training Data Accuracy---------------------#
right=0
wrong=0
TR_TEST=[]
TR_TEST_OUT   =  sigmoid(np.dot(W_Now, TrainingAddData)) 
for res in TR_TEST_OUT:
    if (res>0.5):
        TR_TEST.append(1)
    else:
        TR_TEST.append(0)
for i,j in zip(TR_TEST, TrainingTarget):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Training Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------ValidationData Accuracy---------------------# 
right=0
wrong=0
VAL_TEST=[] 
VAL_TEST_OUT  = sigmoid(np.dot(W_Now, ValDataAdd))
for res in VAL_TEST_OUT:
    if (res>0.5):
        VAL_TEST.append(1)
    else:
        VAL_TEST.append(0)
for i,j in zip(VAL_TEST, ValDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Validation Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------TestingData Accuracy---------------------#
right=0
wrong=0
TEST =[]
TEST_OUT =  sigmoid(np.dot(W_Now, TestDataAdd))  
for res in TEST_OUT:
    if (res>0.5):
        TEST.append(1)
    else:
        TEST.append(0)
for i,j in zip(TEST, TestDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Training Accuracy: 100.0
Validation Accuracy: 97.74436090225564
Testing Accuracy: 98.24561403508771


## Neural Network Solution for Feature Subtraction

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils
import numpy as np

TrainingTarget1 = np_utils.to_categorical(np.array(TrainingTarget),2)

Using TensorFlow backend.


In [36]:
input_size = 472
drop_out = 0.2
first_dense_layer_nodes  = 944
second_dense_layer_nodes = 2

def get_model():
    model = Sequential()
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(first_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.summary()
    
    model.compile(optimizer='AdaDelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [37]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 944)               446512    
_________________________________________________________________
activation_1 (Activation)    (None, 944)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 944)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 944)               892080    
_________________________________________________________________
activation_2 (Activation)    (None, 944)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 944)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1890      
__________

In [38]:
validation_data_split = 0
num_epochs = 1000
model_batch_size = 64
tb_batch_size = 32
early_patience = 100
tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')
history = model.fit(np.transpose(TrainingSubtractData)
                    , TrainingTarget1
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb])


Epoch 1/1000
3200/3200 [==============================] - 4s 1ms/step - loss: 0.7110 - acc: 0.5181
Epoch 2/1000
 192/3200 [>.............................] - ETA: 2s - loss: 0.6906 - acc: 0.5156

D:\MS\1stSem\Anaconda\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


3200/3200 [==============================] - 2s 727us/step - loss: 0.6881 - acc: 0.5409
Epoch 3/1000
3200/3200 [==============================] - 2s 582us/step - loss: 0.6803 - acc: 0.5772
Epoch 4/1000
3200/3200 [==============================] - 2s 556us/step - loss: 0.6451 - acc: 0.6616
Epoch 5/1000
3200/3200 [==============================] - 2s 538us/step - loss: 0.4270 - acc: 0.8244
Epoch 6/1000
3200/3200 [==============================] - 2s 538us/step - loss: 0.2023 - acc: 0.9244
Epoch 7/1000
3200/3200 [==============================] - 2s 520us/step - loss: 0.1298 - acc: 0.9534
Epoch 8/1000
3200/3200 [==============================] - 2s 553us/step - loss: 0.0862 - acc: 0.9712
Epoch 9/1000
3200/3200 [==============================] - 2s 545us/step - loss: 0.1042 - acc: 0.9613
Epoch 10/1000
3200/3200 [==============================] - 2s 548us/step - loss: 0.0754 - acc: 0.9728
Epoch 11/1000
3200/3200 [==============================] - 2s 528us/step - loss: 0.0731 - acc: 0.9728
E

3200/3200 [==============================] - 2s 580us/step - loss: 0.0045 - acc: 0.9981
Epoch 83/1000
3200/3200 [==============================] - 2s 535us/step - loss: 0.0037 - acc: 0.9984
Epoch 84/1000
3200/3200 [==============================] - 2s 524us/step - loss: 0.0012 - acc: 1.0000
Epoch 85/1000
3200/3200 [==============================] - 2s 530us/step - loss: 0.0048 - acc: 0.9984
Epoch 86/1000
3200/3200 [==============================] - 2s 529us/step - loss: 0.0019 - acc: 1.0000
Epoch 87/1000
3200/3200 [==============================] - 2s 529us/step - loss: 0.0033 - acc: 0.9994
Epoch 88/1000
3200/3200 [==============================] - 2s 527us/step - loss: 0.0027 - acc: 0.9997
Epoch 89/1000
3200/3200 [==============================] - 2s 534us/step - loss: 0.0021 - acc: 0.9991
Epoch 90/1000
3200/3200 [==============================] - 2s 532us/step - loss: 0.0015 - acc: 0.9997
Epoch 91/1000
3200/3200 [==============================] - 2s 635us/step - loss: 0.0041 - acc: 0

3200/3200 [==============================] - 2s 599us/step - loss: 3.3402e-04 - acc: 1.0000
Epoch 160/1000
3200/3200 [==============================] - 2s 526us/step - loss: 0.0010 - acc: 0.9994
Epoch 161/1000
3200/3200 [==============================] - 2s 555us/step - loss: 2.9450e-04 - acc: 1.0000
Epoch 162/1000
3200/3200 [==============================] - 2s 622us/step - loss: 6.5631e-04 - acc: 0.9997
Epoch 163/1000
3200/3200 [==============================] - 2s 569us/step - loss: 3.2802e-04 - acc: 1.0000
Epoch 164/1000
3200/3200 [==============================] - 2s 535us/step - loss: 1.6060e-04 - acc: 1.0000
Epoch 165/1000
3200/3200 [==============================] - 2s 529us/step - loss: 2.9805e-04 - acc: 1.0000
Epoch 166/1000
3200/3200 [==============================] - 2s 529us/step - loss: 0.0010 - acc: 0.9997
Epoch 167/1000
3200/3200 [==============================] - 2s 543us/step - loss: 3.0019e-04 - acc: 1.0000
Epoch 168/1000
3200/3200 [==============================] - 

...

3200/3200 [==============================] - 2s 679us/step - loss: 8.1770e-05 - acc: 1.0000
Epoch 691/1000
3200/3200 [==============================] - 2s 674us/step - loss: 2.5016e-05 - acc: 1.0000 0s - loss: 2.4602e-05 - acc: 1
Epoch 692/1000
3200/3200 [==============================] - 3s 789us/step - loss: 1.5977e-04 - acc: 1.0000
Epoch 693/1000
3200/3200 [==============================] - 2s 730us/step - loss: 1.6776e-05 - acc: 1.0000
Epoch 694/1000
3200/3200 [==============================] - 2s 763us/step - loss: 1.7904e-05 - acc: 1.0000
Epoch 695/1000
3200/3200 [==============================] - 2s 721us/step - loss: 4.3732e-05 - acc: 1.0000
Epoch 696/1000
3200/3200 [==============================] - 2s 727us/step - loss: 7.1360e-06 - acc: 1.0000
Epoch 697/1000
3200/3200 [==============================] - 2s 614us/step - loss: 5.4454e-06 - acc: 1.0000
Epoch 698/1000
3200/3200 [==============================] - 2s 638us/step - loss: 1.1966e-05 - acc: 1.0000
Epoch 699/1000
3200/3

3200/3200 [==============================] - 2s 523us/step - loss: 5.4812e-05 - acc: 1.0000
Epoch 767/1000
3200/3200 [==============================] - 2s 510us/step - loss: 5.3311e-05 - acc: 1.0000
Epoch 768/1000
3200/3200 [==============================] - 2s 509us/step - loss: 1.6562e-05 - acc: 1.0000
Epoch 769/1000
3200/3200 [==============================] - 2s 514us/step - loss: 2.8090e-05 - acc: 1.0000
Epoch 770/1000
3200/3200 [==============================] - 2s 515us/step - loss: 6.8796e-04 - acc: 0.9997
Epoch 771/1000
3200/3200 [==============================] - 2s 521us/step - loss: 5.3660e-05 - acc: 1.0000
Epoch 772/1000
3200/3200 [==============================] - 2s 726us/step - loss: 2.5023e-05 - acc: 1.0000
Epoch 773/1000
3200/3200 [==============================] - 2s 746us/step - loss: 1.8983e-05 - acc: 1.0000
Epoch 774/1000
3200/3200 [==============================] - 3s 965us/step - loss: 1.4304e-05 - acc: 1.0000
Epoch 775/1000
3200/3200 [==========================

3200/3200 [==============================] - 2s 758us/step - loss: 4.1555e-06 - acc: 1.0000
Epoch 843/1000
3200/3200 [==============================] - 2s 734us/step - loss: 1.1789e-04 - acc: 1.0000
Epoch 844/1000
3200/3200 [==============================] - 2s 678us/step - loss: 1.4995e-04 - acc: 1.0000
Epoch 845/1000
3200/3200 [==============================] - 3s 796us/step - loss: 9.1685e-06 - acc: 1.0000
Epoch 846/1000
3200/3200 [==============================] - 3s 849us/step - loss: 1.2707e-04 - acc: 1.0000
Epoch 847/1000
3200/3200 [==============================] - 3s 797us/step - loss: 2.0348e-05 - acc: 1.0000
Epoch 848/1000
3200/3200 [==============================] - 3s 850us/step - loss: 1.9612e-05 - acc: 1.0000
Epoch 849/1000
3200/3200 [==============================] - 3s 813us/step - loss: 4.8390e-06 - acc: 1.0000
Epoch 850/1000
3200/3200 [==============================] - 3s 898us/step - loss: 3.4589e-05 - acc: 1.0000
Epoch 851/1000
3200/3200 [==========================

3200/3200 [==============================] - 3s 875us/step - loss: 1.3626e-04 - acc: 1.0000
Epoch 919/1000
3200/3200 [==============================] - 3s 1ms/step - loss: 1.8695e-05 - acc: 1.0000
Epoch 920/1000
3200/3200 [==============================] - 2s 747us/step - loss: 1.1144e-05 - acc: 1.0000
Epoch 921/1000
3200/3200 [==============================] - 3s 854us/step - loss: 4.3511e-06 - acc: 1.0000
Epoch 922/1000
3200/3200 [==============================] - 2s 761us/step - loss: 1.2312e-05 - acc: 1.0000
Epoch 923/1000
3200/3200 [==============================] - 3s 928us/step - loss: 1.0667e-04 - acc: 1.0000
Epoch 924/1000
3200/3200 [==============================] - 3s 816us/step - loss: 2.5718e-04 - acc: 1.0000
Epoch 925/1000
3200/3200 [==============================] - 3s 963us/step - loss: 6.9350e-06 - acc: 1.0000
Epoch 926/1000
3200/3200 [==============================] - 3s 1ms/step - loss: 1.6383e-05 - acc: 1.0000A: 2s 
Epoch 927/1000
3200/3200 [========================

3200/3200 [==============================] - 3s 824us/step - loss: 3.7233e-05 - acc: 1.0000
Epoch 993/1000
3200/3200 [==============================] - 3s 949us/step - loss: 8.2760e-06 - acc: 1.0000 2s -
Epoch 994/1000
3200/3200 [==============================] - 3s 875us/step - loss: 6.0885e-05 - acc: 1.0000
Epoch 995/1000
3200/3200 [==============================] - 3s 885us/step - loss: 1.1199e-05 - acc: 1.0000
Epoch 996/1000
3200/3200 [==============================] - 3s 976us/step - loss: 1.3950e-05 - acc: 1.0000
Epoch 997/1000
3200/3200 [==============================] - 3s 919us/step - loss: 4.3925e-06 - acc: 1.0000
Epoch 998/1000
3200/3200 [==============================] - 3s 1ms/step - loss: 2.9390e-05 - acc: 1.0000
Epoch 999/1000
3200/3200 [==============================] - 3s 962us/step - loss: 7.4434e-05 - acc: 1.0000
Epoch 1000/1000
3200/3200 [==============================] - 3s 1ms/step - loss: 2.2733e-05 - acc: 1.0000


In [39]:
wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TrainingSubtractData),TrainingTarget):
    y = model.predict(np.array(i).reshape(-1,472))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Training Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []
for i,j in zip(np.transpose(ValDataSubtract),ValDataAct):
    y = model.predict(np.array(i).reshape(-1,472))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Validation Accuracy: " + str(right/(right+wrong)*100))


wrong   = 0
right   = 0
predictedTestLabel = []
for i,j in zip(np.transpose(TestDataSubtract),TestDataAct):
    y = model.predict(np.array(i).reshape(-1,472))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Testing Accuracy: " + str(right/(right+wrong)*100))


Errors: 0  Correct :3200
Training Accuracy: 100.0
Errors: 9  Correct :390
Validation Accuracy: 97.74436090225564
Errors: 12  Correct :387
Testing Accuracy: 96.99248120300751


## Neural Network Solution for Feature Addition

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils
import numpy as np

TrainingTarget1 = np_utils.to_categorical(np.array(TrainingTarget),2)

In [41]:
input_size = 935
drop_out = 0.2
first_dense_layer_nodes  = 256
second_dense_layer_nodes = 2

def get_model_add():
    model = Sequential()
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(first_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.summary()
    
    model.compile(optimizer='AdaDelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [42]:
model = get_model_add()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               239616    
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
__________

In [43]:
validation_data_split = 0
num_epochs = 1000
model_batch_size = 128
tb_batch_size = 32
early_patience = 10
tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')
history = model.fit(np.transpose(TrainingAddData)
                    , TrainingTarget1
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb])

Epoch 1/1000
3200/3200 [==============================] - 0s 156us/step - loss: 0.6600 - acc: 0.6031
Epoch 2/1000
2048/3200 [==================>...........] - ETA: 0s - loss: 0.5464 - acc: 0.6328

D:\MS\1stSem\Anaconda\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


3200/3200 [==============================] - 0s 89us/step - loss: 0.4722 - acc: 0.7141
Epoch 3/1000
3200/3200 [==============================] - 0s 86us/step - loss: 0.1319 - acc: 0.9734
Epoch 4/1000
3200/3200 [==============================] - 0s 82us/step - loss: 0.0756 - acc: 0.9869
Epoch 5/1000
3200/3200 [==============================] - 0s 122us/step - loss: 0.0674 - acc: 0.9866
Epoch 6/1000
3200/3200 [==============================] - 0s 121us/step - loss: 0.0599 - acc: 0.9872
Epoch 7/1000
3200/3200 [==============================] - 0s 106us/step - loss: 0.0508 - acc: 0.9875
Epoch 8/1000
3200/3200 [==============================] - 0s 117us/step - loss: 0.0521 - acc: 0.9862
Epoch 9/1000
3200/3200 [==============================] - 0s 108us/step - loss: 0.0489 - acc: 0.9875
Epoch 10/1000
3200/3200 [==============================] - 0s 103us/step - loss: 0.0488 - acc: 0.9872
Epoch 11/1000
3200/3200 [==============================] - 0s 119us/step - loss: 0.0426 - acc: 0.9878
Epoc

3200/3200 [==============================] - 0s 79us/step - loss: 0.0060 - acc: 0.9984
Epoch 83/1000
3200/3200 [==============================] - 0s 92us/step - loss: 0.0037 - acc: 0.9991
Epoch 84/1000
3200/3200 [==============================] - 0s 83us/step - loss: 0.0047 - acc: 0.9984
Epoch 85/1000
3200/3200 [==============================] - 0s 82us/step - loss: 0.0076 - acc: 0.9969
Epoch 86/1000
3200/3200 [==============================] - 0s 74us/step - loss: 0.0036 - acc: 0.9991
Epoch 87/1000
3200/3200 [==============================] - 0s 75us/step - loss: 0.0039 - acc: 0.9994
Epoch 88/1000
3200/3200 [==============================] - 0s 81us/step - loss: 0.0049 - acc: 0.9984
Epoch 89/1000
3200/3200 [==============================] - 0s 71us/step - loss: 0.0049 - acc: 0.9978
Epoch 90/1000
3200/3200 [==============================] - 0s 71us/step - loss: 0.0050 - acc: 0.9984
Epoch 91/1000
3200/3200 [==============================] - 0s 70us/step - loss: 0.0027 - acc: 0.9994
Epoc

3200/3200 [==============================] - 0s 74us/step - loss: 3.3132e-04 - acc: 1.0000: 0s - loss: 3.0791e-04 - acc: 1.000
Epoch 161/1000
3200/3200 [==============================] - 0s 74us/step - loss: 4.9793e-04 - acc: 1.0000
Epoch 162/1000
3200/3200 [==============================] - 0s 124us/step - loss: 7.0865e-04 - acc: 1.0000
Epoch 163/1000
3200/3200 [==============================] - 0s 103us/step - loss: 8.1872e-04 - acc: 0.9997
Epoch 164/1000
3200/3200 [==============================] - 0s 88us/step - loss: 5.8389e-04 - acc: 1.0000
Epoch 165/1000
3200/3200 [==============================] - 0s 98us/step - loss: 5.0493e-04 - acc: 1.0000
Epoch 166/1000
3200/3200 [==============================] - 0s 80us/step - loss: 5.1174e-04 - acc: 1.0000
Epoch 167/1000
3200/3200 [==============================] - 0s 88us/step - loss: 7.4891e-04 - acc: 1.0000
Epoch 168/1000
3200/3200 [==============================] - 0s 80us/step - loss: 0.0018 - acc: 0.9994
Epoch 169/1000
3200/3200 [=

...

3200/3200 [==============================] - 0s 105us/step - loss: 2.9844e-05 - acc: 1.0000
Epoch 691/1000
3200/3200 [==============================] - 0s 112us/step - loss: 4.9379e-04 - acc: 0.9997
Epoch 692/1000
3200/3200 [==============================] - 0s 119us/step - loss: 3.8876e-05 - acc: 1.0000
Epoch 693/1000
3200/3200 [==============================] - 0s 119us/step - loss: 4.6072e-05 - acc: 1.0000
Epoch 694/1000
3200/3200 [==============================] - 0s 106us/step - loss: 4.2271e-05 - acc: 1.0000
Epoch 695/1000
3200/3200 [==============================] - 0s 114us/step - loss: 4.0155e-05 - acc: 1.0000
Epoch 696/1000
3200/3200 [==============================] - 0s 107us/step - loss: 1.8177e-04 - acc: 1.0000
Epoch 697/1000
3200/3200 [==============================] - 0s 122us/step - loss: 8.5060e-05 - acc: 1.0000
Epoch 698/1000
3200/3200 [==============================] - 0s 142us/step - loss: 2.2985e-05 - acc: 1.0000
Epoch 699/1000
3200/3200 [==========================

3200/3200 [==============================] - 0s 156us/step - loss: 3.4128e-05 - acc: 1.0000
Epoch 766/1000
3200/3200 [==============================] - 0s 127us/step - loss: 2.9150e-05 - acc: 1.0000
Epoch 767/1000
3200/3200 [==============================] - 0s 110us/step - loss: 2.0177e-05 - acc: 1.0000
Epoch 768/1000
3200/3200 [==============================] - 0s 103us/step - loss: 3.1998e-05 - acc: 1.0000
Epoch 769/1000
3200/3200 [==============================] - 0s 124us/step - loss: 9.1720e-06 - acc: 1.0000
Epoch 770/1000
3200/3200 [==============================] - 0s 139us/step - loss: 2.2586e-05 - acc: 1.0000
Epoch 771/1000
3200/3200 [==============================] - 0s 131us/step - loss: 1.2705e-05 - acc: 1.0000
Epoch 772/1000
3200/3200 [==============================] - 0s 117us/step - loss: 3.6541e-05 - acc: 1.0000
Epoch 773/1000
3200/3200 [==============================] - 0s 97us/step - loss: 3.6090e-05 - acc: 1.0000
Epoch 774/1000
3200/3200 [===========================

3200/3200 [==============================] - 0s 89us/step - loss: 1.2667e-05 - acc: 1.0000
Epoch 842/1000
3200/3200 [==============================] - 0s 102us/step - loss: 6.7348e-05 - acc: 1.0000
Epoch 843/1000
3200/3200 [==============================] - 0s 108us/step - loss: 5.1260e-05 - acc: 1.0000 0s - loss: 5.8393e-05 - acc: 1.
Epoch 844/1000
3200/3200 [==============================] - 0s 88us/step - loss: 2.0870e-04 - acc: 1.0000
Epoch 845/1000
3200/3200 [==============================] - 0s 120us/step - loss: 4.0584e-05 - acc: 1.0000 0s - loss: 4.6541e-05 - acc: 1.00
Epoch 846/1000
3200/3200 [==============================] - 0s 101us/step - loss: 2.0363e-05 - acc: 1.0000
Epoch 847/1000
3200/3200 [==============================] - 0s 101us/step - loss: 3.1817e-05 - acc: 1.0000
Epoch 848/1000
3200/3200 [==============================] - 0s 100us/step - loss: 2.1923e-05 - acc: 1.0000
Epoch 849/1000
3200/3200 [==============================] - 0s 81us/step - loss: 9.0545e-06 - a

3200/3200 [==============================] - 0s 97us/step - loss: 1.9832e-05 - acc: 1.0000
Epoch 917/1000
3200/3200 [==============================] - 0s 88us/step - loss: 1.3178e-05 - acc: 1.0000
Epoch 918/1000
3200/3200 [==============================] - 0s 83us/step - loss: 2.0953e-05 - acc: 1.0000
Epoch 919/1000
3200/3200 [==============================] - 0s 123us/step - loss: 3.3818e-05 - acc: 1.0000
Epoch 920/1000
3200/3200 [==============================] - 0s 104us/step - loss: 9.3979e-06 - acc: 1.0000
Epoch 921/1000
3200/3200 [==============================] - 0s 108us/step - loss: 5.4060e-05 - acc: 1.0000
Epoch 922/1000
3200/3200 [==============================] - 0s 100us/step - loss: 1.5701e-05 - acc: 1.0000
Epoch 923/1000
3200/3200 [==============================] - 0s 98us/step - loss: 2.1031e-05 - acc: 1.0000
Epoch 924/1000
3200/3200 [==============================] - 0s 131us/step - loss: 1.0763e-04 - acc: 1.0000
Epoch 925/1000
3200/3200 [==============================

3200/3200 [==============================] - 0s 69us/step - loss: 7.1848e-06 - acc: 1.0000
Epoch 994/1000
3200/3200 [==============================] - 0s 68us/step - loss: 1.0644e-04 - acc: 1.0000
Epoch 995/1000
3200/3200 [==============================] - 0s 73us/step - loss: 3.9932e-05 - acc: 1.0000
Epoch 996/1000
3200/3200 [==============================] - 0s 75us/step - loss: 4.0959e-05 - acc: 1.0000
Epoch 997/1000
3200/3200 [==============================] - 0s 72us/step - loss: 3.3268e-05 - acc: 1.0000
Epoch 998/1000
3200/3200 [==============================] - 0s 73us/step - loss: 1.2866e-05 - acc: 1.0000
Epoch 999/1000
3200/3200 [==============================] - 0s 73us/step - loss: 2.7681e-05 - acc: 1.0000
Epoch 1000/1000
3200/3200 [==============================] - 0s 73us/step - loss: 1.7372e-05 - acc: 1.0000


In [44]:
wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TrainingAddData),TrainingTarget):
    y = model.predict(np.array(i).reshape(-1,935))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Training Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []
for i,j in zip(np.transpose(ValDataAdd),ValDataAct):
    y = model.predict(np.array(i).reshape(-1,935))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Validation Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TestDataAdd),TestDataAct):
    y = model.predict(np.array(i).reshape(-1,935))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Errors: 0  Correct :3200
Training Accuracy: 100.0
Errors: 3  Correct :396
Validation Accuracy: 99.24812030075188
Errors: 4  Correct :395
Testing Accuracy: 98.99749373433583
